In [2]:
from __future__ import print_function

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from __init__ import *
from datamanage import DataIO
from advection2d import Grid2d, Simulation
from pdfsolver import PdfGrid
from Learning import PDElearn

Set up input variables for solver and range of computation. 

In [ ]:
problem = "gaussian"
tmax = .7
C = 0.2
k = 0.4
power = 2

xmin = -4.0
xmax = 4.0
ymin = -4.0
ymax = 4.0

params = [0.0, 0.3, 0.0, 0.3, 0]

ng = 3
nx = 400
ny = 400


Create grid for u(x, t) and run monte carlo simulations

In [3]:

err = []

# no limiting
gg = Grid2d(nx, ny, ng, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
sg = Simulation(gg, C=C, k=k, power=power)
sg.init_cond(problem, params)
ainit = sg.grid.a.copy()

print('starting to evolve')
tt, atot = sg.evolve(tmax=tmax)


starting to evolve
 34%|███▍      | 240/707 [02:56<05:42,  1.36it/s]


KeyboardInterrupt: 

In [ ]:
Create 

In [ ]:
mxl = 100
mxr = 100
myl = 100
myr = 100

fu_xUt = atot[gg.ilox+mxl: gg.ihix+1-mxr, gg.iloy+myl: gg.ihiy+1-myr, :]
fu_Uxt = np.transpose(fu_xUt, (1, 0, 2)) # Or np.transpose(fu, (2, 1, 0))
xx = gg.x[gg.ilox+mxl:gg.ihix+1-mxr]
uu = gg.y[gg.iloy+myl:gg.ihiy+1-myr]

U, X = np.meshgrid(uu, xx, indexing='ij')

In [ ]:
# Plotting

tidx = -1

for i in [0, tidx]:
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(U, X, fu_Uxt[:, :, i], cmap=cm.coolwarm, linewidth=0, antialiased=False)
    ax.set_xlabel('U')
    ax.set_ylabel('x')
    ax.set_zlabel('PDF')
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    fig.colorbar(surf, shrink=0.5, aspect=5)


fig = plt.figure()

pd = 10
n = 1
xidxmid = int((len(xx)-1)/2)
xidxvec = [xidxmid-int(n/2)+i*pd for i in range(n)]
c= ['k', 'r', 'b']
for i, xidx in enumerate(xidxvec):
    plt.plot(uu, fu_Uxt[:, xidx, 0], '--', color=c[i])
    plt.plot(uu, fu_Uxt[:, xidx, tidx], color=c[i])
plt.xlabel('U')
plt.ylabel('fu(U; x_0, t)')
plt.legend(['fu(U) t = 0', 'fu(U) t = %3.2f'%(tt[-1])])

plt.savefig('fuU_advection_reaction')

fig = plt.figure()

uidxmid = int((len(uu)-1)/2)
plt.plot(xx, fu_Uxt[uidxmid, :, 0], '--', color=c[i])
plt.plot(xx, fu_Uxt[uidxmid, :, tidx], color=c[i])
plt.xlabel('x')
plt.ylabel('fu(U_0; x, t)')
plt.legend(['fu(x) t = 0', 'fu(x) t = %3.2f'%(tt[-1])])

plt.savefig('fux_advection_reaction')

plt.show()

In [ ]:
# Save
case = 'advection_reaction'
D = DataIO(case=case)

gridvars = {'u': [uu[0], uu[-1], (uu[-1]-uu[0])/len(uu)], 't': [tt[0], tt[-1], (tt[-1]-tt[0])/len(tt)], 'x':[xx[0], xx[-1], (xx[-1]-xx[0])/len(xx)]}
ICparams = {'u0':'gaussian', 
            'fu0':'gaussian',
            'params': params,
            'k': k,
            'reaction u^': power,
            'distribution': 'PDF'}

solution = {'fu': fu_Uxt, 'gridvars': gridvars}
metadata = {'ICparams': ICparams, 'gridvars': gridvars} 

savename = D.saveSolution(solution, metadata)


In [ ]:
# LEARN


loadnamenpy = savename + '.npy'
#loadnamenpy = "advection_reaction_2562.npy"
    
case = '_'.join(loadnamenpy.split('_')[:2])
dataman = DataIO(case) 
fu, gridvars, ICparams = dataman.loadSolution(loadnamenpy)

print(loadnamenpy)

#fu = fu_Uxt

#Make fu smaller (in time)
tt = np.linspace(gridvars['t'][0], gridvars['t'][1], round( (gridvars['t'][1] - gridvars['t'][0]) / gridvars['t'][2] ))
period = 4
indexes = np.array([i*period for i in range((len(tt))//period)])
ttnew = tt[indexes]
fu = fu[:, :, indexes]
gridvars['t'][1] = ttnew[-1]
gridvars['t'][2] = (ttnew[-1]-ttnew[0])/len(ttnew)


grid = PdfGrid(gridvars)
# Learn 
difflearn = PDElearn(grid=grid, fu=fu, ICparams=ICparams, scase=case, trainratio=0.4, debug=False, verbose=True)
difflearn.fit_sparse(feature_opt='1storder', variableCoef=True, variableCoefBasis='simple_polynomial', variableCoefOrder=2, use_sindy=True, sindy_alpha=0.005, shuffle=False)